# Import Libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported successfully!')

Libraries imported successfully!


## Download and Explore Dataset
The dataset is available for free on the Internet at the dataset link: https://geo.nyu.edu/catalog/nyu_2451_34572
For convenience, the files have been downloaded and placed on the server, so you can simply run the wget command and access the data.

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open(r'C:\Users\Marat Abishev\Downloads\nyu_2451_34572-geojson.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [28]:
neighborhoods_data = newyork_data['features']
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [29]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [30]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of NY City is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of NY City is 40.7127281, -74.0060152.


In [31]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [37]:
import folium

map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='black',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [38]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            
            
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            response = requests.get(url).json()
            results = response["response"]['venues']
           
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [40]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'ZMHWBS0SR12Z3YDYVHJVTZPRK3U1ZP3I2TYQAJ5CU3JUHMB5'
CLIENT_SECRET = 'H3TT0XT3P5TIAFCV1Y2UUVLF42N44DICNKLUELK34H2TKLFR'
VERSION = '20213004'

In [43]:
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_sushi = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=2000, categoryIds='4bf58dd8d48988d1d2941735')
newyork_venues_sushi.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Planet Tokyo,40.886233,-73.909479,Sushi Restaurant
1,Marble Hill,40.876551,-73.91066,Yokohama,40.887214,-73.904708,Sushi Restaurant
2,Marble Hill,40.876551,-73.91066,Sushi Mambo,40.861031,-73.920435,Sushi Restaurant
3,Marble Hill,40.876551,-73.91066,Asian Tokyo,40.890839,-73.898335,Sushi Restaurant
4,Marble Hill,40.876551,-73.91066,Mama Sushi,40.866189,-73.927963,Sushi Restaurant


In [44]:
newyork_venues_sushi.shape

(1625, 7)

In [45]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [65]:
map_newyork_sushi = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_sushi, 'red', map_newyork_sushi)

map_newyork_sushi

In [47]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [48]:
manhattan_grouped = newyork_venues_sushi.groupby('Neighborhood').count()
manhattan_grouped

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,47,47,47,47,47,47
Carnegie Hill,50,50,50,50,50,50
Central Harlem,8,8,8,8,8,8
Chelsea,50,50,50,50,50,50
Chinatown,50,50,50,50,50,50
Civic Center,50,50,50,50,50,50
Clinton,50,50,50,50,50,50
East Harlem,21,21,21,21,21,21
East Village,50,50,50,50,50,50


# Analyze Each Neighborhood

In [49]:
manhattan_onehot = pd.get_dummies(newyork_venues_sushi[['Venue Category']], prefix="", prefix_sep="")

manhattan_onehot['Neighborhood'] = newyork_venues_sushi['Neighborhood'] 
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant,Indian Chinese Restaurant,Japanese Restaurant,Noodle House,Ramen Restaurant,Sandwich Place,Seafood Restaurant,Sushi Restaurant,Vegetarian / Vegan Restaurant
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [50]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant,Indian Chinese Restaurant,Japanese Restaurant,Noodle House,Ramen Restaurant,Sandwich Place,Seafood Restaurant,Sushi Restaurant,Vegetarian / Vegan Restaurant
0,Battery Park City,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.106383,0.021277,0.00,0.00,0.00,0.872340,0.000000
1,Carnegie Hill,0.040000,0.00,0.000000,0.00,0.020000,0.000000,0.000000,0.020000,0.140000,0.000000,0.00,0.00,0.00,0.760000,0.020000
2,Central Harlem,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,1.000000,0.000000
3,Chelsea,0.020000,0.02,0.000000,0.00,0.020000,0.000000,0.000000,0.000000,0.100000,0.000000,0.02,0.00,0.00,0.760000,0.060000
4,Chinatown,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.120000,0.020000,0.00,0.00,0.00,0.860000,0.000000
5,Civic Center,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.100000,0.020000,0.00,0.00,0.00,0.880000,0.000000
6,Clinton,0.020000,0.02,0.020000,0.00,0.000000,0.000000,0.000000,0.000000,0.140000,0.000000,0.02,0.00,0.02,0.720000,0.040000
7,East Harlem,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.047619,0.047619,0.047619,0.000000,0.00,0.00,0.00,0.857143,0.000000
8,East Village,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.120000,0.000000,0.00,0.00,0.00,0.860000,0.020000
9,Financial District,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.111111,0.022222,0.00,0.00,0.00,0.866667,0.000000


In [51]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [52]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Sushi Restaurant,Japanese Restaurant,Noodle House,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
1,Carnegie Hill,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Fish Market,Indian Chinese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Grocery Store
2,Central Harlem,Sushi Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant,Indian Chinese Restaurant,Japanese Restaurant
3,Chelsea,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Bakery,Fish Market,Ramen Restaurant,Chinese Restaurant,Deli / Bodega,Grocery Store
4,Chinatown,Sushi Restaurant,Japanese Restaurant,Noodle House,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant


# Cluster Neighborhoods

In [53]:
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

kmeans.labels_[0:10] 

array([0, 1, 2, 1, 0, 0, 1, 0, 0, 0])

In [54]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,2,Sushi Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant,Indian Chinese Restaurant,Japanese Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,0,Sushi Restaurant,Japanese Restaurant,Noodle House,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,2,Sushi Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant,Indian Chinese Restaurant,Japanese Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,2,Sushi Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant,Indian Chinese Restaurant,Japanese Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2,Sushi Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant,Indian Chinese Restaurant,Japanese Restaurant


In [55]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Claster1

In [57]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Sushi Restaurant,Japanese Restaurant,Noodle House,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
7,East Harlem,Sushi Restaurant,Hawaiian Restaurant,Indian Chinese Restaurant,Japanese Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store
19,East Village,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
20,Lower East Side,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
21,Tribeca,Sushi Restaurant,Japanese Restaurant,Noodle House,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
22,Little Italy,Sushi Restaurant,Japanese Restaurant,Noodle House,Vegetarian / Vegan Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store
23,Soho,Sushi Restaurant,Japanese Restaurant,Noodle House,Vegetarian / Vegan Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store
27,Gramercy,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store
28,Battery Park City,Sushi Restaurant,Japanese Restaurant,Noodle House,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
29,Financial District,Sushi Restaurant,Japanese Restaurant,Noodle House,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant


Claster2

In [58]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Roosevelt Island,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Deli / Bodega,Seafood Restaurant,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Fish Market,Grocery Store
14,Clinton,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Ramen Restaurant,Seafood Restaurant,Deli / Bodega,Fish Market
15,Midtown,Sushi Restaurant,Japanese Restaurant,Seafood Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Ramen Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market
16,Murray Hill,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Seafood Restaurant,Ramen Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market
17,Chelsea,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Bakery,Fish Market,Ramen Restaurant,Chinese Restaurant,Deli / Bodega,Grocery Store
30,Carnegie Hill,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Fish Market,Indian Chinese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Grocery Store
33,Midtown South,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Fish Market,Ramen Restaurant,Seafood Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega
34,Sutton Place,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Seafood Restaurant,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store
35,Turtle Bay,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Seafood Restaurant,Vegetarian / Vegan Restaurant,Bakery,Ramen Restaurant,Chinese Restaurant,Deli / Bodega,Fish Market
36,Tudor City,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Seafood Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market


Claster3

In [59]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Sushi Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant,Indian Chinese Restaurant,Japanese Restaurant
2,Washington Heights,Sushi Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant,Indian Chinese Restaurant,Japanese Restaurant
3,Inwood,Sushi Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant,Indian Chinese Restaurant,Japanese Restaurant
4,Hamilton Heights,Sushi Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant,Indian Chinese Restaurant,Japanese Restaurant
5,Manhattanville,Sushi Restaurant,Hawaiian Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store,Indian Chinese Restaurant,Japanese Restaurant
6,Central Harlem,Sushi Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant,Indian Chinese Restaurant,Japanese Restaurant


Claster4

In [60]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Grocery Store,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Hawaiian Restaurant
10,Lenox Hill,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Seafood Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store
12,Upper West Side,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Chinese Restaurant,Grocery Store,Hawaiian Restaurant,Bakery,Deli / Bodega,Fish Market,Indian Chinese Restaurant
13,Lincoln Square,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Chinese Restaurant,Grocery Store,Sandwich Place,Seafood Restaurant,Vegetarian / Vegan Restaurant,Bakery,Deli / Bodega
25,Manhattan Valley,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Hawaiian Restaurant,Indian Chinese Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store


Claster5

In [61]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Yorkville,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Fish Market,Indian Chinese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Grocery Store
18,Greenwich Village,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Fish Market,Noodle House,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Grocery Store
24,West Village,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Fish Market,Noodle House,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Grocery Store
26,Morningside Heights,Sushi Restaurant,Japanese Restaurant,Hawaiian Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store,Indian Chinese Restaurant
37,Stuyvesant Town,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Fish Market,Grocery Store,Hawaiian Restaurant
38,Flatiron,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Fish Market,Asian Restaurant,Bakery,Chinese Restaurant,Deli / Bodega,Grocery Store,Hawaiian Restaurant
